<a href="https://colab.research.google.com/github/oliverwsd/Deep-Portfolio/blob/master/LTSM_50_ae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Run some setup code for this notebook.
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import copy

from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras import regularizers
from keras.constraints import maxnorm
from keras.models import load_model
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler  
from collections import defaultdict

matplotlib.rcParams[ 'figure.figsize' ] = ( 20, 6 )


#packages for modelling
import tensorflow.compat.v1 as tfo
import tensorflow as tf
import random as rd
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import GridSearchCV
from scipy.stats import reciprocal

#packages for plotting
%matplotlib inline
import matplotlib.dates as md
import pydot

#packages for extraction and saving data
import os
import glob as glob
import csv

#packages for date handling
import datetime
from time import time

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def normalize_data(df):
    '''
    Use the preprocessing object MinMaxScaler from sklearn library to normalize the data.
    The out dataframe has the same index and column
    '''
    x = df.values 
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_n = pd.DataFrame(x_scaled, index=df.index, columns=df.columns)
    return df_n

In [10]:
from io import StringIO
import csv
# realized variance 
x_data = 'https://raw.githubusercontent.com/oliverwsd/Deep-Portfolio/master/sp500_stock_selected.csv'
y_data = 'https://raw.githubusercontent.com/oliverwsd/Deep-Portfolio/master/sp500_index_rv.csv'
# read csv
stock_data = pd.read_csv(x_data,index_col=0)
index_data = pd.read_csv(y_data, index_col=0)

In [7]:
stock_data.head()

,PH,APH,FISV,IQV,USB,PNR,FIS,AEP,TROW,MOS,ALXN,GILD,TWTR,EIX,EBAY,STX,HSIC,PAYC,SJM,NEM,AKAM,TTWO,NFLX,EA,DISCK,AMZN,HRB,REGN,FLT,CTL,WST,EFX,IDXX,RHI,DISCA,NOW,HBI,ADSK,DVA,ZBRA,WMT,AOS,UAA,BBY,VZ,CERN,VRTX,FFIV,DLTR,ABC
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016/1/5,0.000279,0.000127,0.000021,0.000001,0.000023,0.000073,3.997583e-05,0.000067,0.000017,0.000018,0.000001,1.606078e-04,0.000828,1.150576e-07,0.000139,0.000003,0.000012,0.000965,0.000056,0.000001,0.000031,0.000283,0.000447,0.000313,0.000019,0.000025,0.000082,0.000218,0.000779,0.000023,0.000163,0.000036,0.000099,1.053307e-05,0.000005,6.938600e-07,0.000606,0.000002,4.266400e-05,0.000647,0.000551,0.000031,0.000038,0.000262,0.000186,1.894576e-06,0.000021,0.000018,0.000217,0.000211
2016/1/6,0.000178,0.000573,0.000058,0.000031,0.000232,0.000538,3.453040e-04,0.000014,0.000514,0.004622,0.000020,1.086400e-04,0.000599,4.402732e-05,0.000122,0.001977,0.000104,0.000012,0.000054,0.000057,0.000903,0.000463,0.007919,0.000384,0.000046,0.000003,0.000052,0.000417,0.000926,0.000118,0.000044,0.000064,0.000051,7.252658e-04,0.000052,9.731828e-04,0.000005,0.000201,1.956146e-04,0.001663,0.000099,0.000400,0.000012,0.001543,0.000082,6.322907e-05,0.000099,0.000245,0.000045,0.000237
2016/1/7,0.000292,0.001347,0.000132,0.000559,0.000368,0.000715,2.850177e-08,0.000134,0.000733,0.001227,0.002931,1.698815e-03,0.002946,2.463022e-05,0.000764,0.000037,0.000276,0.000243,0.000297,0.000257,0.002155,0.002292,0.000722,0.000374,0.000997,0.001587,0.000078,0.001177,0.001015,0.000418,0.000495,0.001350,0.000458,1.265302e-05,0.000956,4.886067e-04,0.000274,0.003801,6.326768e-05,0.002151,0.000530,0.002509,0.000561,0.000087,0.000030,7.521563e-07,0.003760,0.001174,0.000678,0.001053
2016/1/8,0.000330,0.000228,0.000060,0.000002,0.000203,0.000333,3.265447e-04,0.000002,0.000659,0.000604,0.001328,4.308193e-06,0.000194,2.947367e-08,0.000005,0.003558,0.000085,0.004396,0.000005,0.002127,0.000003,0.000006,0.000787,0.000114,0.000760,0.000002,0.000175,0.000428,0.000035,0.000108,0.000097,0.000864,0.000104,6.266185e-04,0.000424,3.713456e-04,0.002349,0.000275,5.462470e-07,0.000007,0.000537,0.001313,0.001538,0.001872,0.000095,3.379697e-04,0.001419,0.000011,0.000071,0.000034
2016/1/11,0.000004,0.000060,0.000022,0.000067,0.000016,0.000009,1.914867e-04,0.000076,0.000029,0.002078,0.001545,2.687280e-07,0.000277,2.453156e-04,0.000182,0.001388,0.000691,0.000082,0.000034,0.003263,0.003187,0.000174,0.001001,0.000288,0.000036,0.000305,0.000010,0.000499,0.000843,0.000039,0.000019,0.000016,0.000084,5.210388e-08,0.000048,2.740063e-05,0.002510,0.000010,4.255340e-05,0.000044,0.000113,0.000688,0.004839,0.000270,0.000033,6.512040e-04,0.003970,0.000224,0.000031,0.001650


In [11]:
index_data.head()

,RV
Date,
2016/1/5,4.040000e-06
2016/1/6,1.742970e-04
2016/1/7,5.753190e-04
2016/1/8,1.187560e-04
2016/1/11,7.270000e-07
